In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from scipy import stats
from statistics import stdev
import datetime
from datetime import timedelta
from collections import Counter
import cx_Oracle
import psycopg2
import seaborn as sns
import json
import pickle
#from fitter import Fitter
from warnings import filterwarnings
filterwarnings('ignore')

Get for each AS_ID the percentage of PG3 and PG4

In [2]:
shop_arts_pg3_pg4 = pd.read_csv('shop_arts_pg3_pg4.dsv', sep=';', low_memory=False)

In [3]:
shop_arts_pg3_pg4.head()

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK
0,64,50,276,1
1,70,401,1592,1
2,114,48,404,1
3,138,1400,1418,1
4,150,18,394,1


In [4]:
shop_arts_pg3_pg4.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2522274 entries, 0 to 2522273
Data columns (total 4 columns):
AS_ID                     int64
PG3                       int64
PG4                       int64
PG_PRICE_PCT_OVER_PACK    object
dtypes: int64(3), object(1)
memory usage: 77.0+ MB


In [5]:
shop_arts_pg3_pg4['PG_PRICE_PCT_OVER_PACK'] = [float(str(i).replace(',','.')) 
                                               for i in shop_arts_pg3_pg4['PG_PRICE_PCT_OVER_PACK'].values]

If we check the length of the table and the different AS_ID included we can see how some AS_ID are configured as the combination of several AP_IDs. 

In [6]:
len(shop_arts_pg3_pg4), shop_arts_pg3_pg4['AS_ID'].nunique()

(2522274, 2058092)

In [7]:
shop_arts_pg3_pg4[shop_arts_pg3_pg4['AS_ID'] == 1638057]

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK
519138,1638057,2068,152944,0.241
1386235,1638057,301,152946,0.226
1701658,1638057,252,152818,0.533


In [8]:
shop_arts_pg3_pg4[shop_arts_pg3_pg4['AS_ID'] == 1882486]

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK
606668,1882486,251,255,0.988
1000598,1882486,16,35657,0.012


Check how to add brand name to each PG4 for the brand specific concept...

In [9]:
pg4_brands = pd.read_csv('pg4_brands.dsv', sep=';', low_memory=False)

In [11]:
pg4_brands.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2006 entries, 0 to 2005
Data columns (total 2 columns):
PPR_N_LEVEL4_ID      2006 non-null int64
PPR_V_LEVEL4_DESC    2006 non-null object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [28]:
pg4_brands['BRAND'] = [i.split(' - ')[1] if len(i.split('-')) > 1 else 'unknown' for i in pg4_brands['PPR_V_LEVEL4_DESC'].values]

In [29]:
pg4_brands[pg4_brands['BRAND'] == 'unknown']

,PPR_N_LEVEL4_ID,PPR_V_LEVEL4_DESC,BRAND
521,0,*NOT DEFINED,unknown


In [30]:
pg4_brands.sample(10)

,PPR_N_LEVEL4_ID,PPR_V_LEVEL4_DESC,BRAND
1821,17624,17624 - Defu Biofutter,Defu Biofutter
638,9796,9796 - Pferdepellets,Pferdepellets
1279,1548,1548 - Katzenhalsband,Katzenhalsband
59,905,905 - Hängematten,Hängematten
1871,597,597 - ASK MARTIN STOLZ,ASK MARTIN STOLZ
1147,1395,1395 - Küchen-Ideen,Küchen-Ideen
1960,107606,107606 - Farmina,Farmina
1925,48652,48652 - Haut & Haare,Haut & Haare
875,3195,3195 - Teichfischfutter,Teichfischfutter
1005,72159,72159 - ★ Cosma,★ Cosma


In [22]:
len(pg4_brands), pg4_brands['BRAND'].nunique()

(2006, 1535)

In [34]:
shop_arts_pg3_pg4_brand = shop_arts_pg3_pg4.merge(pg4_brands, how='left', left_on='PG4', right_on= 'PPR_N_LEVEL4_ID')

In [35]:
shop_arts_pg3_pg4_brand.tail(25)

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,PPR_N_LEVEL4_ID,PPR_V_LEVEL4_DESC,BRAND
2522249,2521951,251,313067,1.000,313067,313067 - Canine Caviar - NEU,Canine Caviar
2522250,2521963,34,72159,1.000,72159,72159 - ★ Cosma,★ Cosma
2522251,2521969,20,1426,0.039,1426,1426 - Quietscher,Quietscher
2522252,2522021,401,2263,1.000,2263,2263 - Kräuter & Raufutter,Kräuter & Raufutter
2522253,2522104,34,72002,1.000,72002,72002 - Getrocknet & natürlich,Getrocknet & natürlich
2522254,2522139,34,72159,1.000,72159,72159 - ★ Cosma,★ Cosma
2522255,2522172,2068,2069,1.000,2069,2069 - Animonda Carny,Animonda Carny
2522256,2522210,251,36043,1.000,36043,36043 - IAMS,IAMS
2522257,2522213,252,286,1.000,286,286 - Pro Plan,Pro Plan
2522258,2522231,257304,257327,1.000,257327,257327 - Chips & Snacks,Chips & Snacks


Before joining the tables we need to compress all info regarding every AS_ID in one row just to avoid duplicates.

In [36]:
arts_pg3_pg4_brand = shop_arts_pg3_pg4_brand.groupby('AS_ID', as_index=False).agg({
                                                                        'PG3': lambda x: list(x),
                                                                        'PG4': lambda x: list(x),
                                                                        'PG_PRICE_PCT_OVER_PACK': lambda x: list(x),
                                                                        'BRAND': lambda x: list(x)
                                                                      })

In [37]:
arts_pg3_pg4_brand[arts_pg3_pg4_brand['AS_ID'] == 1638057]

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
1200360,1638057,"[2068, 301, 252]","[152944, 152946, 152818]","[0.241, 0.226, 0.533]","[★ Greenwoods, ★ Greenwoods, ★ Greenwoods]"


In [39]:
arts_pg3_pg4_brand.sample(50)

,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
1783577,2241577,[20],[96500],[1.0],[Kuschelspielzeug]
1804721,2263247,[2068],[3155],[1.0],[Schesir]
1622928,2075433,[857],[2136],[1.0],[Grau]
789775,1190338,[18],[101],[1.0],[Kauknochen & Kaurollen]
638120,1014714,[251],[255],[1.0],[Eukanuba]
152429,414999,[251],[1308],[1.0],[Royal Canin Club / Selection]
1082214,1515450,[2068],[2072],[1.0],[Gourmet Gold]
1821859,2280758,[251],[2304],[1.0],[Josera]
1501141,1948573,[2068],[2069],[1.0],[Animonda Carny]
1614468,2066603,[29],[128],[1.0],[Haubentoiletten]


In [40]:
len(arts_pg3_pg4_brand), arts_pg3_pg4_brand['AS_ID'].nunique()

(2058092, 2058092)

In [42]:
#As it takes some time, let's write the output dataframe using pickle
#arts_pg3_pg4_brand.to_pickle("./arts_pg3_pg4_brand.pkl")

In [3]:
arts_pg3_pg4_brand = pd.read_pickle("./arts_pg3_pg4_brand.pkl")

Valid orders within 1 year in ES Shop (from 01/10/2017 to 30/09/2018) with a TC mapping

In [4]:
data = pd.read_csv('one_year_es_shop_orders.dsv', sep=';', low_memory=False)

In [5]:
data.head()

,OP_ID,O_DATE,O_ID,O_K_ID,TC_ID,OP_AS_ID,OP_AMOUNT,OP_PRICE,O_GWERT,OSP_KO_COUNT_NET
0,415830551,01/11/2017 10:17:20,97470440,16869155,133572,1112102,1,"43,99","43,99",6
1,415852740,01/11/2017 11:17:55,97475660,17393486,116812,426151,1,"4,49","76,75",5
2,415852741,01/11/2017 11:17:55,97475660,17393486,116812,2174195,1,"9,99","76,75",5
3,415852742,01/11/2017 11:17:55,97475660,17393486,116812,1294727,1,"3,29","76,75",5
4,415831331,01/11/2017 10:19:43,97470694,3197737,2601861,1276255,1,"4,99","97,35",73


In [6]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4382371 entries, 0 to 4382370
Data columns (total 10 columns):
OP_ID               int64
O_DATE              object
O_ID                int64
O_K_ID              int64
TC_ID               int64
OP_AS_ID            int64
OP_AMOUNT           int64
OP_PRICE            object
O_GWERT             object
OSP_KO_COUNT_NET    int64
dtypes: int64(7), object(3)
memory usage: 334.3+ MB


In [7]:
data[data['OP_ID'].duplicated()]

,OP_ID,O_DATE,O_ID,O_K_ID,TC_ID,OP_AS_ID,OP_AMOUNT,OP_PRICE,O_GWERT,OSP_KO_COUNT_NET


In [8]:
data['O_DATE'] = pd.to_datetime(data['O_DATE'], format="%d/%m/%Y %H:%M:%S")

In [9]:
data['OP_PRICE'] = [float(str(i).replace(',','.')) for i in data['OP_PRICE'].values]
data['O_GWERT'] = [float(str(i).replace(',','.')) for i in data['O_GWERT'].values]

In [10]:
data['OP_TOT_SALES'] = data['OP_AMOUNT'] * data['OP_PRICE']

In [11]:
data['MAX_SALES_O_POS'] = data['OP_PRICE'].groupby(data['O_ID']).transform('max')
data['IS_LEAD_AS'] = (data['OP_PRICE'] == data['MAX_SALES_O_POS']) * 1 * data['OP_AS_ID']
data['LEAD_AS_ID'] = data['IS_LEAD_AS'].groupby(data['O_ID']).transform('max')

In [12]:
data = data.merge(arts_pg3_pg4_brand, left_on='OP_AS_ID', right_on='AS_ID', how='left')

In [13]:
data.tail()

,OP_ID,O_DATE,O_ID,O_K_ID,TC_ID,OP_AS_ID,OP_AMOUNT,OP_PRICE,O_GWERT,OSP_KO_COUNT_NET,OP_TOT_SALES,MAX_SALES_O_POS,IS_LEAD_AS,LEAD_AS_ID,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
4382366,517448779,2018-10-29 21:56:56,122273191,21699666,16254094,1860497,1,9.99,39.22,1,9.99,12.99,0,2478332,1860497.0,[9],[74],[1.0],[Hundedecken]
4382367,517448777,2018-10-29 21:56:56,122273191,21699666,16254094,1483373,1,3.29,39.22,1,3.29,12.99,0,2478332,1483373.0,[18],[250153],[1.0],[Würstchen & Fleischbällchen]
4382368,517902410,2018-10-31 10:57:02,122384642,21714869,16268163,1,1,-4.80,44.18,1,-4.80,47.99,0,1352515,NaN,NaN,NaN,NaN,NaN
4382369,517902409,2018-10-31 10:57:02,122384642,21714869,16268163,1352515,1,47.99,44.18,1,47.99,47.99,1352515,1352515,1352515.0,[252],[683],[1.0],[Royal Canin]
4382370,517897237,2018-10-31 10:42:04,122383348,12351462,218144,2408627,1,49.99,99.98,15,49.99,49.99,2408627,2408627,2408627.0,[251],[682],[1.0],[Royal Canin Size]


In [14]:
data[data['AS_ID'].isnull()]

,OP_ID,O_DATE,O_ID,O_K_ID,TC_ID,OP_AS_ID,OP_AMOUNT,OP_PRICE,O_GWERT,OSP_KO_COUNT_NET,OP_TOT_SALES,MAX_SALES_O_POS,IS_LEAD_AS,LEAD_AS_ID,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
11,415852760,2017-11-01 11:17:58,97475666,9499754,7520318,6,1,-2.20,41.79,26,-2.20,43.99,0,1284911,NaN,NaN,NaN,NaN,NaN
16,415833197,2017-11-01 10:24:56,97471048,7066411,5777176,6,1,-2.85,92.14,44,-2.85,94.99,0,1977933,NaN,NaN,NaN,NaN,NaN
17,415833209,2017-11-01 10:25:03,97471051,10007491,7866814,1476857,3,0.00,70.79,78,0.00,42.99,0,1838628,NaN,NaN,NaN,NaN,NaN
18,415833210,2017-11-01 10:25:03,97471051,10007491,7866814,2191227,1,0.00,70.79,78,0.00,42.99,0,1838628,NaN,NaN,NaN,NaN,NaN
22,415787506,2017-11-01 06:20:26,97459845,17447813,12844945,6,1,-2.35,44.64,4,-2.35,46.99,0,2203258,NaN,NaN,NaN,NaN,NaN
25,415830372,2017-11-01 10:16:57,97470359,16204085,11988106,6,1,-2.12,40.26,6,-2.12,36.39,0,1868399,NaN,NaN,NaN,NaN,NaN
30,415833431,2017-11-01 10:26:08,97471184,4346457,110817,6,1,-1.20,38.77,33,-1.20,21.99,0,1303805,NaN,NaN,NaN,NaN,NaN
40,415785077,2017-11-01 03:32:42,97459270,12157538,9172804,6,1,-10.22,194.15,15,-10.22,46.99,0,1509219,NaN,NaN,NaN,NaN,NaN
52,415803108,2017-11-01 08:43:29,97463793,9177366,7300443,1,1,-7.00,132.98,12,-7.00,69.99,0,1304656,NaN,NaN,NaN,NaN,NaN
53,415803106,2017-11-01 08:43:29,97463793,9177366,7300443,1855238,2,0.00,132.98,12,0.00,69.99,0,1304656,NaN,NaN,NaN,NaN,NaN


In [15]:
data[data['O_ID'] == 97475660]

,OP_ID,O_DATE,O_ID,O_K_ID,TC_ID,OP_AS_ID,OP_AMOUNT,OP_PRICE,O_GWERT,OSP_KO_COUNT_NET,OP_TOT_SALES,MAX_SALES_O_POS,IS_LEAD_AS,LEAD_AS_ID,AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
1,415852740,2017-11-01 11:17:55,97475660,17393486,116812,426151,1,4.49,76.75,5,4.49,47.99,0,1179482,426151.0,[20],[465],[1.0],[KONG]
2,415852741,2017-11-01 11:17:55,97475660,17393486,116812,2174195,1,9.99,76.75,5,9.99,47.99,0,1179482,2174195.0,[20],[350],[1.0],[Intelligenzspielzeug]
3,415852742,2017-11-01 11:17:55,97475660,17393486,116812,1294727,1,3.29,76.75,5,3.29,47.99,0,1179482,1294727.0,[18],[250154],[1.0],[Softe & halbfeuchte Snacks]
20,415852743,2017-11-01 11:17:55,97475660,17393486,116812,1179482,1,47.99,76.75,5,47.99,47.99,1179482,1179482,1179482.0,[251],[17798],[1.0],[Royal Canin Breed (Rasse)]
600,415852744,2017-11-01 11:17:55,97475660,17393486,116812,2051207,1,10.99,76.75,5,10.99,47.99,0,1179482,2051207.0,[1400],[1412],[1.0],[Haut & Fell]


In [16]:
# See how to compute PG3-PG4 % when grouping at the order level

In [19]:
#Operations to do when grouping at order level
agg_ops = {'O_DATE': min,
          'O_K_ID': min, 
          'TC_ID': min,
          'OSP_KO_COUNT_NET': min,
          'O_GWERT': max,
          'OP_AS_ID': lambda x: list(x),
          'OP_PRICE': lambda x: list(x),
          'OP_AMOUNT': lambda x: list(x),
          'OP_TOT_SALES': lambda x: list(x),
          'LEAD_AS_ID': max,
          'PG3': lambda x: list(x),
          'PG4': lambda x: list(x),
          'PG_PRICE_PCT_OVER_PACK': lambda x: list(x),
          'BRAND': lambda x: list(x)
          }

In [20]:
orders = data.groupby('O_ID', as_index=False).agg(agg_ops)

In [21]:
orders.sample(10)

,O_ID,O_DATE,O_K_ID,TC_ID,OSP_KO_COUNT_NET,O_GWERT,OP_AS_ID,OP_PRICE,OP_AMOUNT,OP_TOT_SALES,LEAD_AS_ID,PG3,PG4,PG_PRICE_PCT_OVER_PACK,BRAND
324767,104917489,2018-02-18 20:06:11,18342952,390022,6,50.39,"[2190948, 2301008, 1451652, 667545, 1]","[0.0, 55.99, 0.0, 0.0, -5.6]","[1, 1, 1, 1, 1]","[0.0, 55.99, 0.0, 0.0, -5.6]",2301008,"[nan, [251], nan, nan, nan]","[nan, [35825], nan, nan, nan]","[nan, [1.0], nan, nan, nan]","[nan, [Taste of the Wild], nan, nan, nan]"
336970,105184264,2018-02-22 13:27:50,19500711,14188421,1,23.18,"[1037893, 2316353, 2347785, 1, 2, 2316352]","[8.99, 1.99, 3.49, -1.25, 3.99, 1.99]","[1, 2, 1, 1, 1, 2]","[8.99, 3.98, 3.49, -1.25, 3.99, 3.98]",1037893,"[[2068], [2068], [2068], nan, nan, [2068]]","[[2073], [17963], [2073], nan, nan, [17963]]","[[1.0], [1.0], [1.0], nan, nan, [1.0]]","[[Hill’s Science Plan], [Royal Canin], [Hill’s..."
418941,107045355,2018-03-20 20:21:49,9435656,241712,41,67.89,"[1597718, 6]","[69.99, -2.1]","[1, 1]","[69.99, -2.1]",1597718,"[[251], nan]","[[17798], nan]","[[1.0], nan]","[[Royal Canin Breed (Rasse)], nan]"
999068,120327602,2018-10-02 22:35:19,13005184,9842355,6,54.11,"[1704475, 1558747, 322412, 1839734, 6]","[29.99, 9.99, 2.99, 13.99, -2.85]","[1, 1, 1, 1, 1]","[29.99, 9.99, 2.99, 13.99, -2.85]",1704475,"[[251], [45], [45], [45], nan]","[[283], [397], [397], [397], nan]","[[1.0], [1.0], [1.0], [1.0], nan]","[[Pro Plan], [Degufutter], [Degufutter], [Degu..."
31027,98149312,2017-11-11 10:43:51,11425145,8814949,12,50.99,[1830667],[50.99],[1],[50.99],1830667,[[251]],[[35825]],[[1.0]],[[Taste of the Wild]]
512880,109134811,2018-04-21 11:03:29,20024488,14696278,2,57.89,"[1811636, 2]","[54.99, 2.9]","[1, 1]","[54.99, 2.9]",1811636,"[[9], nan]","[[30936], nan]","[[1.0], nan]","[[Gesundheitsbetten], nan]"
307573,104505054,2018-02-12 13:38:36,16811091,3975,4,49.99,"[2248160, 1, 2000874]","[49.99, 0.0, 0.0]","[1, 1, 1]","[49.99, 0.0, 0.0]",2248160,"[[857, 251], nan, nan]","[[180309, 180361], nan, nan]","[[0.207, 0.793], nan, nan]","[[★ Wolf of Wilderness, ★ Wolf of Wilderness],..."
53256,98672110,2017-11-19 15:08:20,18575646,12630638,1,95.97,"[1910600, 2136819, 1311078]","[21.99, 29.99, 43.99]","[1, 1, 1]","[21.99, 29.99, 43.99]",1311078,"[[38], [2068], [252]]","[[72369], [2071], [683]]","[[1.0], [1.0], [1.0]]","[[für Gelenke & Knochen], [Felix], [Royal Canin]]"
530546,109538095,2018-04-27 09:51:42,17003862,138264,9,42.97,"[1224462, 1305864, 879094]","[2.99, 22.99, 16.99]","[1, 1, 1]","[2.99, 22.99, 16.99]",1305864,"[[36], [252], [252]]","[[679], [683], [261925]]","[[1.0], [1.0], [1.0]]","[[Mäuse & Bälle], [Royal Canin], [Royal Canin ..."
183,97462586,2017-11-01 08:17:28,5790034,4411099,19,39.74,"[1, 1308664, 6]","[-10.0, 16.99, -1.23]","[1, 3, 1]","[-10.0, 50.97, -1.23]",1308664,"[nan, [2068], nan]","[nan, [17963], nan]","[nan, [1.0], nan]","[nan, [Royal Canin], nan]"


Percentage of each PG4 per order...

In [ ]:
sorted([(pg4, np.round(sum([pg_[2] * share_ for share_, pg_info_ in list(zip(ord_data['OP_TOT_SALES'], ord_data['PG3_PG4_INFO_'])) 
                            for pg_ in pg_info_ if pg4==pg_[1]]), 3)) for pg4 in set([pg[1] for share, pg_info in list(zip(ord_data['O_POS_SALES_SHARE'], ord_data['PG3_PG4_INFO_'])) 
                            for pg in pg_info])], key= lambda x: x[1], reverse=True)

In [ ]:
def sales_per_pg4(df):
    sorted([(pg4, np.round(sum([pg_[2] * share_ for share_, pg_info_ in list(zip(df['O_POS_SALES_SHARE'], df['PG3_PG4_INFO_'])) 
                            for pg_ in pg_info_ if pg4==pg_[1]]), 3)) for pg4 in set([pg[1] for share, pg_info in list(zip(df['O_POS_SALES_SHARE'], df['PG3_PG4_INFO_'])) 
                            for pg in pg_info])], key= lambda x: x[1], reverse=True)